# Deep Learning Workshop 5 - Image Retraining in TensorFlow
2019/09/18

**[ Reference ]**
1. `TensorFlow - Tutorials: Image retraining`, "**How to Retrain an Image Classifier for New Categories**". https://www.tensorflow.org/hub/tutorials/image_retraining#using_the_retrained_model

2. "**TensorFlow Hub Installation**", https://www.tensorflow.org/hub/installation

3. `imagenet/inception_v3/feature_vector`, https://tfhub.dev/google/imagenet/inception_v3/feature_vector/3
>    + **Inception V3** is a neural network architecture for image classification, originally published by
        + Christian Szegedy, Vincent Vanhoucke, Sergey Ioffe, Jonathon Shlens, Zbigniew Wojna: "**Rethinking the Inception Architecture for Computer Vision**", 2015. https://arxiv.org/abs/1512.00567

## How to Retrain an Image Classifier for New Categories

>+ Modern image recognition models have millions of parameters. 
+ Training them from scratch requires a lot of labeled training data and a lot of computing power (hundreds of GPU-hours or more). 
+ **Transfer learning** is a technique that shortcuts much of this by taking a piece of a model that has already been trained on a related task and reusing it in a new model. 

+ In this tutorial, we will **reuse the feature extraction capabilities from powerful image classifiers trained on `ImageNet`** and **simply train a new classification layer on top**.


+ Though it's not as good as training the full model, this is surprisingly effective for many applications, works with moderate amounts of training data (thousands, not millions of labeled images), and can be **run in as little as thirty minutes on a laptop without a GPU**. 


+ This tutorial will show you how to run the example script on your own images, and will explain some of the options you have to help control the training process.

----------------------------
## `TensorFlow Hub`
+ `TensorFlow Hub` is a way to share pretrained model components.
+ **This tutorial uses `TensorFlow Hub` to ingest pre-trained pieces of models, or modules as they are called.**
+ For starters, we will use the **image feature extraction module** **[Ref 3]** with the `Inception V3` architecture trained on `ImageNet`, and come back later to further options, including `NASNet/PNASNet`, as well as `MobileNet V1 and V2`.


### Installation of `TensorFlow Hub` **[Ref. 2]**
+ **Before you start, you need to install the PIP package `tensorflow-hub`, along with a sufficiently recent version of TensorFlow (for example, tensorflow 1.14.0).** 
+ See `TensorFlow Hub`'s installation instructions for details in **Ref. 2**.

> **On Anaconda Prompt**:
>
>   `pip install "tensorflow>=1.7.0"`    
    `pip install tensorflow-hub`
>    
>    **[NOTE]:** 
>  + TensorFlow Hub depends on bug fixes and enhancements not present in TensorFlow releases prior to 1.7. 
>  + **You must install or upgrade your TensorFlow package to at least 1.7 to use TensorFlow Hub.**
----------------------------

## Training on Flowers

### On `Anaconda Prompt`, run the following commands :

#### < STEP 1 >  To get the dataset of flower photos, run these commands:
------------------------
`cd C:\`

`curl -LO http://download.tensorflow.org/example_images/flower_photos.tgz`

`tar xzf flower_photos.tgz`

------------------------

#### < STEP 2 > Once you have the images, you can download the example code from GitHub : 
------------------------
`mkdir example_code`

`cd example_code`

`curl -LO https://github.com/tensorflow/hub/raw/master/examples/image_retraining/retrain.py`

------------------------

#### < STEP 3 > In the simplest cases the retrainer can then be run like this (takes about half an hour):
------------------
`python retrain.py --image_dir flower_photos`

------------------


> #### [NOTE]: The script has many other options. You can get a full listing with:
  `python retrain.py -h`

> + This script loads the pre-trained module and trains a new classifier on top for the flower photos you've downloaded. 
+ None of the flower species were in the original ImageNet classes the full network was trained on. 
+ **The magic of transfer learning is that lower layers that have been trained to distinguish between some objects can be reused for many recognition tasks without any alteration.**

## Bottlenecks

+ The first phase analyzes all the images on disk and calculates and caches the bottleneck values for each of them. 


+ **'Bottleneck'** is an informal term we often use *for the layer just before the final output layer that actually does the classification*. (TensorFlow Hub calls this an "image feature vector".) 


+ This penultimate layer has been trained to output a set of values that's good enough for the classifier to use to distinguish between all the classes it's been asked to recognize. 
    + That means it has to be a meaningful and compact summary of the images, since it has to contain enough information for the classifier to make a good choice in a very small set of values. 
    
    
+ **The reason our final layer retraining can work on new classes is that it turns out the kind of information needed to distinguish between all the 1,000 classes in ImageNet is often also useful to distinguish between new kinds of objects.**


+ Because every image is reused multiple times during training and calculating each bottleneck takes a significant amount of time, *it speeds things up to cache these bottleneck values on disk so they don't have to be repeatedly recalculated*. 
    + **By default they're stored in the /tmp/bottleneck directory, and if you rerun the script they'll be reused so you don't have to wait for this part again.**

## Training

+ Once the bottlenecks are complete, the actual training of the top layer of the network begins. You'll see a series of step outputs, each one showing training accuracy, validation accuracy, and the cross entropy.


+ If the train accuracy is high but the validation accuracy remains low, that means the network is overfitting and memorizing particular features in the training images that aren't helpful more generally. 
    + Cross entropy is a loss function which gives a glimpse into how well the learning process is progressing. 
    + The training's objective is to make the loss as small as possible, so you can tell if the learning is working by keeping an eye on whether the loss keeps trending downwards, ignoring the short-term noise.
    

+ By default this script will run 4,000 training steps. 
    + Each step chooses ten images at random from the training set, finds their bottlenecks from the cache, and feeds them into the final layer to get predictions. 
    + Those predictions are then compared against the actual labels to update the final layer's weights through the back-propagation process. 
    + As the process continues you should see the reported accuracy improve, and after all the steps are done, a final test accuracy evaluation is run on a set of images kept separate from the training and validation pictures. 
    + This test evaluation is the best estimate of how the trained model will perform on the classification task. 
    + You should see an accuracy value of between 90% and 95%, though the exact value will vary from run to run since there's randomness in the training process. 
    + This number is based on the percent of the images in the test set that are given the correct label after the model is fully trained.

## Visualizing the Retraining with TensorBoard

`tensorboard --logdir /tmp/retrain_logs`

## Using the Retrained Model

+ The script will write out the new model trained on your categories to **/tmp/output_graph.pb**, and a text file containing the labels to **/tmp/output_labels.txt**. 
    + The new model contains both the TF-Hub module inlined into it, and the new classificiation layer. 
    + The two files are both in a format that the C++ and Python image classification examples can read in, so you can start using your new model immediately. 
    + Since you've replaced the top layer, you will need to specify the new name in the script, for example with the flag **--output_layer=final_result** if you're using label_image.


+ Here's an example of how to run the label_image example with your retrained graphs. 
+ By convention, all TensorFlow Hub modules accept image inputs with color values in the fixed range [0,1], so you do not need to set the **--input_mean** or **--input_std flags**.

`curl -LO https://github.com/tensorflow/tensorflow/raw/master/tensorflow/examples/label_image/label_image.py`

`python label_image.py --graph=/tmp/output_graph.pb --labels=/tmp/output_labels.txt --input_layer=Placeholder --output_layer=final_result --image=./flower_photos/daisy/21652746_cc379e0eea_m.jpg`

+ You should see a list of flower labels, in most cases with daisy on top (though each retrained model may be slightly different). 
    + **You can replace the --image parameter with your own images to try those out**.


+ If you'd like to use the retrained model in your own Python program, then the above label_image script is a reasonable starting point. 
    + The label_image directory also contains C++ code which you can use as a template to integrate tensorflow with your own applications.


+ **If you find the default `Inception V3` module is too large or slow for your application, take a look at `the Other Model Architectures` section below for options to speed up and slim down your network.**

## Training on Your Own Categories

+ If you've managed to get the script working on the flower example images, you can start looking at teaching it to recognize categories you care about instead. 


+ In theory all you'll need to do is point it at a set of sub-folders, each named after one of your categories and containing only images from that category. 
    + If you do that and pass the root folder of the subdirectories as the argument to **--image_dir**, the script should train just like it did for the flowers.


+ Here's what the folder structure of the flowers archive looks like, to give you and example of the kind of layout the script is looking for:

![title](./Fig_1_Data_for_Retraining.png)


+ In practice it may take some work to get the accuracy you want. I'll try to guide you through some of the common problems you might encounter below.
    + Creating a Set of Training Images (https://www.tensorflow.org/hub/tutorials/image_retraining#creating_a_set_of_training_images)
    + Training Steps (https://www.tensorflow.org/hub/tutorials/image_retraining#training_steps)
    + Distortions (https://www.tensorflow.org/hub/tutorials/image_retraining#distortions)
    + Hyper-parameters (https://www.tensorflow.org/hub/tutorials/image_retraining#hyper-parameters)
    + Training, Validation, and Testing Sets (https://www.tensorflow.org/hub/tutorials/image_retraining#training_validation_and_testing_sets)

## Other Model Architectures
> + By default the script uses an image feature extraction module with a pretrained instance of the `Inception V3` architecture. 
+ This was a good place to start because it provides high accuracy results with moderate running time for the retraining script.

###  Further options of a TensorFlow Hub module
+ Training with a different module is easy: Just pass the **`--tfhub_module`** flag with the module URL, for example:

`python retrain.py --image_dir ~/flower_photos --tfhub_module https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/3`

+ This will create a 9 MB model file in **/tmp/output_graph.pb** with a model that uses the baseline version of **MobileNet V2**. 
+ Opening the module URL in a browser will take you to the module documentation.

### As before, you can use all of the retrained models with label_image.py. You will need to specify the image size that your model expects, for example:

`python label_image.py \
--graph=/tmp/output_graph.pb --labels=/tmp/output_labels.txt \
--input_layer=Placeholder \
--output_layer=final_result \
--input_height=224 --input_width=224 \
--image=$HOME/flower_photos/daisy/21652746_cc379e0eea_m.jpg`